<a href="https://colab.research.google.com/github/alywonder/smighty/blob/master/smighty/notebooks/base_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The template notebook with base configurations and setup

## Base setup

### Setup and configuration

In [ ]:
# @title Basic installs
!pip install -q torch
!pip install -q pydantic
!pip install -q typer[all]
!pip install -q python-dotenv

In [ ]:
# @title Basic imports

# @markdown Absolutely run this. We don't repeat these imports in
# other cells, but are used heavily throughout

import torch
import pydantic
import typer
from pydantic import BaseModel, dataclasses
from rich import print
from rich.pretty import pprint, Pretty
from rich.panel import Panel

eprint = lambda obj: pprint(obj, expand_all=True)

In [ ]:
# @title Notebook configuration setup and starting imports
@dataclasses.dataclass
class NotebookConfig:
  """Configuration for the notebook"""
  in_colab: bool = False
  gpu_available: bool = False
  gpu_count: int = 0
  device: str = 'cpu'
  use_dot_env: bool = False

notebook_config = NotebookConfig()

eprint(notebook_config)
# pprint(notebook_config, expand_all=True)
# pprint(notebook_config)
# print(Panel(Pretty(notebook_config)))
# print(Panel(Pretty(notebook_config, expand_all=True)))

In [ ]:
# @title Check and verify colab environment and GPU availability
try:
  import google.colab
  notebook_config.in_colab = True
except:
  notebook_config.in_colab = False

if not torch.cuda.is_available():
  print("[bold red]No GPU available, using the CPU instead.[/]")
  notebook_config.gpu_available = False
  notebook_config.device = 'cpu'
else:
  notebook_config.gpu_available = True
  notebook_config.device = 'cuda:0'
  notebook_config.gpu_count = torch.cuda.device_count()
  !nvidia-smi

In [ ]:
# @title Check for local .env file
import os
from dotenv import load_dotenv, find_dotenv

local_dotenv = find_dotenv()
if local_dotenv:
  notebook_config.use_dot_env = True
  _ = load_dotenv(local_dotenv) # read local .env file

eprint(notebook_config)

In [ ]:
# @title Setup model config
from enum import Enum

# Model class
class ModelFamily(str, Enum):
  openai = 'openai'
  llama = 'llama'
  anthropic = 'anthropic'
  cohere = 'cohere'

# Model names
class OpenAIModels(str, Enum):
  gpt_3_5 = 'gpt-3.5-turbo-1106'
  gpt_4 = 'gpt-4'

class LlamaModels(str, Enum):
  llama_7b = 'meta-llama/Llama-2-7b-hf'
  llama_7b_chat = 'meta-llama/Llama-2-7b-chat'
  codellama_7b = 'codellama/CodeLlama-7b-Python-hf'
  codellama_7b_chat = 'codellama/CodeLlama-7b-Instruct-hf'

class AnthropicModels(str, Enum):
  claude2 = 'claude2'

class CohereModels(str, Enum):
  cohere_small = 'cohere-small'
  cohere_medium = 'cohere-medium'
  cohere_large = 'cohere-large'


@dataclasses.dataclass
class ModelConfig:
  """Configuration for the model"""
  model_family: ModelFamily = ModelFamily.openai
  model_name: OpenAIModels | LlamaModels | AnthropicModels | CohereModels = OpenAIModels.gpt_3_5

model_config = ModelConfig()
eprint(model_config)


In [ ]:
# @title Setup generation config

@dataclasses.dataclass
class GenerationConfig:
  """Configuration for the generation"""
  add_special_tokens: bool = False
  do_sample: bool = True
  max_tokens: int = 1000
  max_new_tokens: int = 200
  num_return_sequences: int = 1
  temperature: float = 0.0
  top_k: int = 10
  top_p: float = 0.95

generation_config = GenerationConfig()
eprint(generation_config)

In [ ]:
# @title Load appropriate keys
# @title Load keys from environment or ask user input


@dataclasses.dataclass
class KeyChain:
  """Store keys"""
  openai_api_key: str = ''
  anthropic_api_key: str = ''
  cohere_api_key: str = ''

key_chain = KeyChain()


if notebook_config.use_dot_env:
  match model_config.model_family:
    case ModelFamily.openai:
      key_chain.openai_api_key = os.environ.get('OPENAI_API_KEY')
    case ModelFamily.anthropic:
      key_chain.anthropic_api_key = os.environ.get('ANTHROPIC_API_KEY')
    case ModelFamily.cohere:
      key_chain.cohere_api_key = os.environ.get('COHERE_API_KEY')

else:
  match model_config.model_family:
    case ModelFamily.openai:
      key_chain.openai_api_key = "Enter OpenAI Key Here" #@param {type:"string"}
    case ModelFamily.anthropic:
      key_chain.anthropic_api_key = "Enter Anthropic Key Here" #@param {type:"string"
    case ModelFamily.cohere:
      key_chain.cohere_api_key = "Enter Cohere Key Here" #@param {type:"string"

eprint(key_chain)